In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'torch'
import torch
import keras
from keras import layers
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from sklearn.datasets import load_breast_cancer

In [ ]:
data = load_breast_cancer()
x = data['data'][:-10]
y = data['target'][:-10]
x_test = data['data'][-10:]
y_test = data['target'][-10:]
print(x.shape, data['feature_names'])
print(y.shape, data['target_names'])

In [ ]:
model = keras.Sequential([
    layers.Input(shape=(30,)),
    layers.Dense(20, activation='tanh'),
    layers.Dropout(rate=0.2),
    layers.Dense(12, activation='tanh'),
    layers.Dropout(rate=0.2),
    layers.Dense(4, activation='tanh'),
    layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["binary_accuracy"])
model.summary()

In [ ]:
history = model.fit(x,y,epochs=100,validation_split=0.1)

In [ ]:
threshold = 0.5

y_pred = np.squeeze(model.predict(x_test))
y_class = np.array([1 if prediction > threshold else 0 for prediction in y_pred])
print(y_class)
print(y_test)